In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('crawl-movies-info/movies-info.csv')

In [14]:
df.values

array([['tt1462758', 0, 0, ..., 1, 0, 0],
       ['tt6491178', 1, 0, ..., 1, 0, 0],
       ['tt2872732', 1, 0, ..., 1, 0, 0],
       ...,
       ['tt3797766', 0, 0, ..., 0, 0, 0],
       ['tt0164279', 0, 0, ..., 0, 0, 0],
       ['tt1723073', 0, 0, ..., 0, 0, 0]], dtype=object)

In [11]:
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer(smooth_idf=True, norm ='l2')
tfidf = transformer.fit_transform(df.values[:,1:].tolist()).toarray()

In [ ]:
import numpy as np
def get_items_rated_by_user(rate_matrix, user_id):
    """
    in each line of rate_matrix, we have infor: user_id, item_id, rating (scores), time_stamp
    we care about the first three values
    return (item_ids, scores) rated by user user_id
    """
    y = rate_matrix[:,0] # all users
    # item indices rated by user_id
    # we need to +1 to user_id since in the rate_matrix, id starts from 1 
    # while index in python starts from 0
    ids = np.where(y == user_id +1)[0] 
    item_ids = rate_matrix[ids, 1] - 1 # index starts from 0 
    scores = rate_matrix[ids, 2]
    return (item_ids, scores)

In [36]:
import numpy as np
from sklearn.linear_model import Ridge
from sklearn import linear_model
 
def predict_rating(movie_ids, scores,alpha=0.01):
    clf = Ridge(alpha=alpha, fit_intercept  = True)
    Xhat = tfidf[movie_ids, :]
    
    clf.fit(Xhat, scores)
    W = clf.coef_
    b = clf.intercept_
    rating = tfidf.dot(W) + b
    return rating

In [21]:
movie_ids=np.array([1,2,3,4,5,6])
scores=np.array([1.0,3.0,5.0,6.0,2.0,8.0])
predict_rating(movie_ids,scores)

array([3.67794815, 1.08623024, 2.98916455, ..., 4.63962796, 4.63962796,
       4.63962796])

In [37]:
# calculate all predicting in training set
train_df = pd.read_csv('crawl-ratings/train_ids.csv')
test_df = pd.read_csv('crawl-ratings/train_ids.csv')
SE=0

for user in train_df.UserIndex.unique():
    train_rating = train_df[train_df.UserIndex==user]
    predict = predict_rating(train_rating.MovieIndex.values,train_rating.Rating.values)
    
    test_rating = test_df[test_df.UserIndex==user]
    SE += np.sum((test_rating.Rating - predict[test_rating.MovieIndex.values])**2)

In [39]:
RMSE = np.sqrt(SE/test_df.shape[0])
RMSE

1.7851635088044007